## Update Group

The API Nomenclature for Power BI Workspaces is "Groups"  Here we're looking at the API used to create a new Group.  This alters the properties of an exsiting workspace.  Note that the calling user must be an administrator, and certain properties can only be set if the Workspace is hosted on a premium capacity.

The API Documentation can be found here: https://learn.microsoft.com/en-us/rest/api/power-bi/groups/update-group

In [ ]:
pip install requests msal

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
from services.aadservice import AadService
aadToken = AadService.get_access_token()

headers =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadToken}

In [ ]:
#This retreives the id of a workspace by name so we don't have hardcoded workspace ids in the code.
import requests
import json
from services.env import const

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/'       
workspaceId = ""

apiResponse = requests.get(apiUrl, headers=headers)
#error handling for createTemporaryUplodadLocation
if apiResponse.status_code != 200:
    description = f'Error creating workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    workspaces = apiResponse["value"]
    for workspace in workspaces:
        if workspace["name"] == const.workspaceName:
            workspaceId = workspace["id"]
            
print(f"WorkspaceId is {workspaceId} ")

In [ ]:
import requests
import json
from services.env import const

apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}'       

#This just changes the name of the workspace
body = {
  "name": const.workspaceName + "-1"
}

apiResponse = requests.patch(apiUrl, headers=headers, data=json.dumps(body))
#error handling for workspace update
if apiResponse.status_code != 200:
    description = f'Error creating workspace:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    print(f"Workspace {workspaceId} updated")